In [6]:
!python -V
!pip -V
!python -c "import sys, pkgutil; print('numpy', pkgutil.find_loader('numpy') is not None); print('torch', pkgutil.find_loader('torch') is not None)"
!pip install --upgrade --no-deps timm pylibjpeg pylibjpeg-libjpeg pylibjpeg-openjpeg
!pip check || true

Python 3.11.13
pip 24.1.2 from /usr/local/lib/python3.11/dist-packages/pip (python 3.11)
numpy True
torch True
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 70.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 40.3 MB/s eta 0:00:0000:01
  Attempting uninstall: timm
    Found existing installation: timm 1.0.19
    Uninstalling timm-1.0.19:
      Successfully uninstalled timm-1.0.19
bigframes 2.12.0 requires google-cloud-bigquery-storage, which is not installed.
pylibjpeg-libjpeg 2.3.0 has requirement numpy<3.0,>=2.0, but you have numpy 1.26.4.
pylibjpeg-openjpeg 2.5.0 has requirement numpy<3.0,>=2.0, but you have numpy 1.26.4.
gensim 4.3.3 has requirement scipy<1.14.0,>=1.7.0, but you have scipy 1.15.3.
datasets 4.1.1 has requirement pyarrow>=21.0.0, but you have pyarrow 19.0.1.
onnx 

In [7]:
import os
import random
import numpy as np
import pandas as pd
from PIL import Image
import pydicom
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.cuda.amp import autocast, GradScaler
import timm
from tqdm import tqdm

In [8]:
DATA_PATH = "/kaggle/input/rsna-2022-cervical-spine-fracture-detection"
train_df = pd.read_csv(os.path.join(DATA_PATH, "train.csv"))
test_df = pd.read_csv(os.path.join(DATA_PATH, "test.csv"))
sample_sub = pd.read_csv(os.path.join(DATA_PATH, "sample_submission.csv"))
TRAIN_IMG_DIR = os.path.join(DATA_PATH, "train_images")

In [9]:
class CervicalSliceDataset(Dataset):
    def __init__(self, df, root, transform=None):
        self.df = df
        self.root = root
        self.transform = transform
        self.study_ids = df["StudyInstanceUID"].unique().tolist()
    def __len__(self):
        return len(self.study_ids)
    def __getitem__(self, idx):
        study = self.study_ids[idx]
        folder = os.path.join(self.root, study)
        files = sorted([f for f in os.listdir(folder) if f.endswith(".dcm")])
        if len(files) == 0:
            raise RuntimeError(f"No DICOM in {folder}")
        chosen = files[len(files)//2]
        path = os.path.join(folder, chosen)
        ds = pydicom.dcmread(path)
        try:
            arr = ds.pixel_array
        except Exception:
            ds.decompress()
            arr = ds.pixel_array
        if arr.ndim == 3:
            arr = arr[0]
        img = Image.fromarray(arr).convert("L")
        if self.transform:
            img = self.transform(img)
        row = self.df[self.df["StudyInstanceUID"]==study].iloc[0]
        labels = torch.zeros(8, dtype=torch.float32)
        labels[0] = row["patient_overall"]
        for i in range(1,8):
            labels[i] = row[f"C{i}"]
        return img, labels


train_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomRotation(10),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])
train_ds = CervicalSliceDataset(train_df, TRAIN_IMG_DIR, transform=train_transforms)
train_loader = DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=2, pin_memory=True)


## Modelo ViT

In [10]:
# Para pruebas cpu y para el final si usare gpu cuda
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=8)
model = model.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5, weight_decay=1e-2)

In [11]:
scaler = GradScaler() if torch.cuda.is_available() else None

def train_epoch(model, loader, optimizer, criterion, device, scaler=None):
    model.train()
    running_loss = 0.0
    for imgs, labels in tqdm(loader):
        imgs = imgs.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)
        optimizer.zero_grad()
        if scaler is not None:
            with autocast():
                out = model(imgs)
                loss = criterion(out, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            out = model(imgs)
            loss = criterion(out, labels)
            loss.backward()
            optimizer.step()
        running_loss += loss.item() * imgs.size(0)
    return running_loss / len(loader.dataset)

for epoch in range(1,4):
    loss = train_epoch(model, train_loader, optimizer, criterion, device, scaler)
    print(f"epoch {epoch} loss {loss:.4f}")

/tmp/ipykernel_37/1633072353.py:1: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler() if torch.cuda.is_available() else None
  0%|          | 0/127 [00:00<?, ?it/s]


RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/tmp/ipykernel_37/3414392125.py", line 19, in __getitem__
    arr = ds.pixel_array
          ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pydicom/dataset.py", line 2193, in pixel_array
    self.convert_pixel_data()
  File "/usr/local/lib/python3.11/dist-packages/pydicom/dataset.py", line 1726, in convert_pixel_data
    self._pixel_array = pixel_array(self, **opts)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pydicom/pixels/utils.py", line 1430, in pixel_array
    return decoder.as_array(
           ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pydicom/pixels/decoders/base.py", line 982, in as_array
    self._validate_plugins(decoding_plugin),
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pydicom/pixels/common.py", line 257, in _validat